# Analyzing Students' Mental Health in SQL

In this live code-along, you'll perform exploratory data analysis on a dataset around mental health of domestic and international students. You'll perform SQL querying to look at how social connectedness and cultural issues affect mental health. Finally, you'll visualize the results of your analysis using the Python Plotly package.

## The Data

This survey was conducted in 2018 at an international Japanese university and the associated study was published in 2019. It was approved by several ethical and regulatory boards.

The study found that international students have a higher risk of mental health difficulties compared to the general population, and that social connectedness and acculturative stress are predictive of depression.

Social connectedness: measure of belonging to a social group or network.

Acculturative stress: stress associated with learning about and intergrating into a new culture.

[See paper for more info, including data description.](https://www.mdpi.com/2306-5729/4/3/124/htm)

[Link to the data.](https://www.mdpi.com/2306-5729/4/3/124/s1)

## Create database connection

-   Databases > Connect Database > PostgreSQL
-   **Database connection name:** Live Training Student MH
-   **Port:** `5432`
-   **Hostname:** `workspacedemodb.datacamp.com`
-   **Database:** `students`
-   **Username:** `students_codealong`
-   **Password:** `students_codealong`

### Inspect the Data

Our data is in one table that includes all of the survey data. There are 50 fields and, according to the paper, 268 records. Each row is a student.

You can check the schema on the left.

1. Check if the data has 268 records.

In [ ]:
-- Count the number of records in the data
select count(*) as total_records
from students;

,total_records
0,286


2. Inspect the dataset to see what the fields look like.

In [ ]:
-- Inspect the data and limit the output to 5 records
select *
from public.students
limit 5;

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Inter,SEA,Male,Grad,24,4,5,Long,3,Average,5,High,,Yes,No,No,No,0,Min,34,23,9,11,8,11,2,27,91,5,5,6,3,2,1,4,1,3,4,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
1,Inter,SEA,Male,Grad,28,5,1,Short,4,High,4,High,,No,No,No,No,2,Min,48,8,7,5,4,3,2,10,39,7,7,7,4,4,4,4,1,1,1,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
2,Inter,SEA,Male,Grad,25,4,6,Long,4,High,4,High,Yes,Yes,No,No,No,2,Min,41,13,4,7,6,4,3,14,51,3,3,3,1,1,2,1,1,1,1,NaN,No,No,No,No,No,No,No,No,No,No,No
3,Inter,EA,Female,Grad,29,5,1,Short,2,Low,3,Average,No,No,No,No,No,3,Min,37,16,10,10,8,6,4,21,75,5,5,5,5,5,2,2,2,4,4,NaN,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No
4,Inter,EA,Female,Grad,28,5,1,Short,1,Low,3,Average,Yes,No,No,No,No,3,Min,37,15,12,5,8,7,4,31,82,5,5,5,2,5,2,5,5,4,4,NaN,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No


3. How many international and domestic students are in the data set?

In [ ]:
-- Count the number of international and domestic students
select inter_dom, count(inter_dom) as count_inter_dom
from public.students
group by inter_dom;

,inter_dom,count_inter_dom
0,Dom,67
1,,18
2,Inter,201


4. Look into the 18 unassigned rows to understand what they could be.

In [ ]:
-- Query the data to see all records where inter_dom is neither 'Dom' nor 'Inter'
SELECT *
FROM public.students
where inter_dom not like 'D%' AND inter_dom not like 'I%'

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,222.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,249.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,203.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,247.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.0,42.0,NaN,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,128.0,137.0,66.0,61.0,30.0,46.0,19.0,65.0,21.0,45.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172.0,54.0,NaN,107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.0,140.0,131.0,202.0,207.0,238.0,222.0,249.0,203.0,247.0,223.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172.0,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,128.0,137.0,66.0,61.0,30.0,46.0,19.0,65.0,21.0,45.0


5. Where are the international students from?

In [ ]:
-- See what Region international students are from
SELECT Region, count(inter_dom) as count_inter_dom
from public.students
where inter_dom = 'Inter'
group by Region

,region,count_inter_dom
0,SA,18
1,EA,48
2,JAP,2
3,Others,11
4,SEA,122


### Understanding the scores

6. Find the minimum, maximum, and average of each of the diagnostic tests (PHQ-9, SCS, ASISS). This information is in the paper, but it's good practice to look this up yourself during analysis.

In [ ]:
-- Find out the basic summary statistics of the diagnostic tests for all students
SELECT
	MIN(todep) as min_phq,
	MAX(todep) as max_phq,
	round(avg(todep),2) as avg_phq,
	MIN(tosc) as min_scs,
	MAX(tosc) as max_scs,
	round(avg(tosc),2) as avg_scs,
	MIN(toas) as min_as,
	MAX(toas) as max_as,
	round(avg(toas),2) as avg_as
FROM public.students

,min_phq,max_phq,avg_phq,min_scs,max_scs,avg_scs,min_as,max_as,avg_as
0,0,25,8.19,8,48,37.47,36,145,72.38


**On your own:**

7. What are the summary statistics for domestic students and international students?

In [ ]:
-- Write a query that looks at the statistics of each student group on one table, remembering to avoid the empty rows
SELECT inter_dom,
	MIN(todep) as min_phq,
	MAX(todep) as max_phq,
	round(avg(todep),2) as avg_phq,
	MIN(tosc) as min_scs,
	MAX(tosc) as max_scs,
	round(avg(tosc),2) as avg_scs,
	MIN(toas) as min_as,
	MAX(toas) as max_as,
	round(avg(toas),2) as avg_as
FROM public.students
where inter_dom in ('Inter', 'Dom')
group by inter_dom
order by inter_dom

,inter_dom,min_phq,max_phq,avg_phq,min_scs,max_scs,avg_scs,min_as,max_as,avg_as
0,Dom,0,23,8.61,8,48,37.64,36,112,62.84
1,Inter,0,25,8.04,11,48,37.42,36,145,75.56


### International Focus

The study found that international students presented a higher risk of having mental health difficulties. Recall, the data is also skewed towards international students. Let's take a closer look at this student group.

8. How does the age of the international student impact the scores?

In [ ]:
-- Find the average scores for each age group of international students, and view them in order
SELECT
	Age,
	round(avg(todep),2) as avg_phq,
	round(avg(tosc),2) as avg_scs,
	round(avg(toas),2) as avg_as
FROM public.students
where inter_dom = 'Inter'
group by age
order by age


,age,avg_phq,avg_scs,avg_as
0,17,4.67,37.33,70.67
1,18,8.75,34.11,80.61
2,19,8.44,37.90,74.10
3,20,7.35,38.21,73.26
4,21,9.23,37.74,75.23
5,22,8.36,38.14,70.43
6,23,9.67,32.00,81.25
7,24,4.67,42.33,74.33
8,25,6.11,37.33,80.78
9,27,10.00,35.00,42.00


**On Your Own**

9. See how another variable may impact the score.

In [ ]:
-- Look into another variable
select stay,
	count(inter_dom) as count_int,
	round(avg(todep),2) as average_phq,
	round(avg(tosc),2) as average_scs,
	round(avg(toas),2) as average_as
from students
where (inter_dom = 'Inter')
group by stay
order by stay desc

,stay,count_int,average_phq,average_scs,average_as
0,10,1,13.00,32.00,50.00
1,8,1,10.00,44.00,65.00
2,7,1,4.00,48.00,45.00
3,6,3,6.00,38.00,58.67
4,5,1,0.00,34.00,91.00
5,4,14,8.57,33.93,87.71
6,3,46,9.09,37.13,78.00
7,2,39,8.28,37.08,77.67
8,1,95,7.48,38.11,72.80


# Interactive plots with Plotly
Check out Introduction to Data Visualization with Plotly in Python to learn more.

In [3]:
# Import plotly packages
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

data = pd.read_csv('student_mh_data.csv')

In [ ]:
-- Make sure the data you want is saved to a variable
SELECT *
FROM students;

,inter_dom,Region,Gender,Academic,Age,Age_cate,Stay,Stay_Cate,Japanese,Japanese_cate,...,Friends_bi,Parents_bi,Relative_bi,Professional_bi,Phone_bi,Doctor_bi,religion_bi,Alone_bi,Others_bi,Internet_bi
0,Inter,SEA,Male,Grad,24.0,4.0,5.0,Long,3.0,Average,...,Yes,Yes,No,No,No,No,No,No,No,No
1,Inter,SEA,Male,Grad,28.0,5.0,1.0,Short,4.0,High,...,Yes,Yes,No,No,No,No,No,No,No,No
2,Inter,SEA,Male,Grad,25.0,4.0,6.0,Long,4.0,High,...,No,No,No,No,No,No,No,No,No,No
3,Inter,EA,Female,Grad,29.0,5.0,1.0,Short,2.0,Low,...,Yes,Yes,Yes,Yes,No,No,No,No,No,No
4,Inter,EA,Female,Grad,28.0,5.0,1.0,Short,1.0,Low,...,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,,,,,NaN,NaN,NaN,,NaN,,...,140,,,,,,,,,
282,,,,,NaN,NaN,NaN,,NaN,,...,131,,,,,,,,,
283,,,,,NaN,NaN,NaN,,NaN,,...,202,,,,,,,,,
284,,,,,NaN,NaN,NaN,,NaN,,...,207,,,,,,,,,


### Histograms

In [4]:
# Create the histogram figure
fig = px.histogram(

# Select the dataframe
    data_frame=data,

# Select the column you want to visualize
    x="ToDep",

# Select the additional column to determine different groups of data
    color="inter_dom",

# Clean up the labels
    title="Distribution of the totla scores from the PHQ-9 test",
    labels={"inter_dom":"Type of Student", "ToDep":"Total PHQ-9 Scores"})

# Show the figure
fig.show()

### Box Plots

In [5]:
# Create the box plot figure
fig = px.box(

# Select the dataframe
    data_frame=data,

# Select the column you want to visualize
    x="ToDep",

# Select the additional column to determine different groups of data
    color="inter_dom",

# Select the additional column to determine what information is shown when you hover over the plot
    hover_data=["Age"],

# Select the option to view all data points
    points="all",

# Clean up the labels
    title="Box plot of the total scores from the PHQ-9 test",
    labels={"inter_dom":"Type of Student", "ToDep":"Total PHQ-9 Scores"})

# Show the figure
fig.show()

### Correlation Plot

In [7]:
# List the columns that are continuous variables
continuous_variables = ['Age', 'Stay', 'Japanese', 'English', 'ToDep', 'ToSC', 'APD', 'AHome', 'APH', 'Afear', 'ACS', 'AGuilt', 'AMiscell', 'ToAS', 'Partner', 'Friends', 'Parents', 'Relative', 'Profess',  'Doctor', 'Reli', 'Alone', 'Others', 'Internet']

# Create a subset dataframe only the columns of the continous variables
data_cont = data[continuous_variables]

# Create a pearson correlation
data_corr = data_cont.corr(method='pearson')

# Build the Heatmap
fig = go.Figure(go.Heatmap(x=data_corr.columns, y=data_corr.columns, z=data_corr.values.tolist(), zmin=-1, zmax=1))

# Adjust the plot size
fig.update_layout(width=900, height=900)

# Show the plot
fig.show()

### Dropdown Interactivity

In [8]:
# Create the figure
fig = go.Figure()

# Write a for loop to loop over the variable you want to have in the drop down
for stu_type in ['Inter', 'Dom']:
    df = data[data.inter_dom == stu_type]
    fig.add_trace(go.Histogram(x=df["ToDep"], name=stu_type))

# Create the dropdown buttons
dropdown_buttons = [
    {'label':'All', 'method':'update', 'args':[{'visible': [True, True]}, {'title': 'All'}]},
    {'label':'International', 'method':'update', 'args':[{'visible': [True, False]}, {'title': 'International'}]},
    {'label':'Domestic', 'method':'update', 'args':[{'visible':[False, True]}, {'title':'Domestic'}]}
]

# Add the dropdown to the figure
fig.update_layout(
    {'updatemenus':[{'type':"dropdown",
         'x': 1.3,
         'y': 0.5,
         'showactive':True,
         'active':0,
         'buttons': dropdown_buttons}]
    }
)

# Show the figure
fig.show()